<a href="https://colab.research.google.com/github/akilesh10113/Time-Series-Analysis-/blob/main/Time_Series_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor

In [62]:
train = pd.read_csv('train.csv', parse_dates=['date'])


In [63]:
train_weekly = (
    train.groupby(['store_nbr', 'family', pd.Grouper(key='date', freq='W')])['sales']
    .sum()
    .reset_index()
)

In [64]:
train_weekly = train_weekly.fillna(0)

In [65]:
train_weekly['year'] = train_weekly['date'].dt.year
train_weekly['month'] = train_weekly['date'].dt.month
train_weekly['weekofyear'] = train_weekly['date'].dt.isocalendar().week

In [66]:
train_weekly['lag_1'] = train_weekly.groupby(['store_nbr','family'])['sales'].shift(1)
train_weekly['lag_2'] = train_weekly.groupby(['store_nbr','family'])['sales'].shift(2)
train_weekly['rolling_mean_3'] = (
    train_weekly.groupby(['store_nbr','family'])['sales'].shift(1).rolling(3).mean()
)


In [67]:
train_weekly = train_weekly.dropna().reset_index(drop=True)

In [68]:
le = LabelEncoder()
train_weekly['family'] = le.fit_transform(train_weekly['family'])

In [69]:
X = train_weekly.drop(columns=['sales', 'date'])
y = train_weekly['sales']

model = XGBRegressor(
    tree_method='hist',
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    random_state=42
)
model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, ...)

In [70]:
preds = model.predict(X)
mae = mean_absolute_error(y, preds)
rmse = np.sqrt(mean_squared_error(y, preds))

print("✅ Model Performance:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

✅ Model Performance:
Mean Absolute Error (MAE): 234.96
Root Mean Squared Error (RMSE): 924.15
